In [1]:
import pandas as pd
import numpy as np

In [4]:
input_file = 'data/ORF3a_data.csv'
tcrs=pd.read_csv(input_file)
refs = pd.read_csv('data/combined_cdr3.csv')


In [13]:
out=[]
with open ('data/MIRA_DATA_FULL.txt','r',encoding='latin') as f:
    for line in f.readlines():
        out.append(line.strip('\n').split('\t'))
        

In [28]:
mira = pd.DataFrame(out[1:],columns=out[0])

In [46]:
cdr3,v,j =[[x.split('+')[i] for x in mira['TCR'].values] for i in range(0,3,1)]

In [48]:
cols=[cdr3,v,j]

In [59]:
for i,col in enumerate(['cdr3','v','j']):
        mira[col]=cols[i]
        if i!=0:
                mira[col]=['*'.join(x.split('-')[:2]) for x in mira[col].values]

In [66]:
mira['HLA-A'].unique()

array(['A*02:01:01', 'A*11:01:01', 'A*02:01', 'A*03:01', 'A*11:01',
       'A*24:02:01', 'A*03:01:01', 'A*01:01', 'A*01:01:01', 'A*24:07:01',
       'A*32:01:01', 'NA', 'A*02:13', 'A*26:02:01', 'A*02:10',
       'A*02:03:01', 'A*32:01'], dtype=object)

In [68]:
l=len(mira)
mira=mira.dropna(subset=['cdr3','v','j','HLA-A'])
print(l-len(mira))


0


In [67]:
mira['Epitope']=['HLA-'+mira.iloc[i]['HLA-A']+'_'+mira.iloc[i]['Peptide']+'_'+mira.iloc[i]['ORF Coverage']+'_SARS-CoV-2' for i in range(len(mira))]

In [85]:
mira=mira.rename(columns={'cdr3':'cdr3.beta',
                          'v':'v.beta',
                          'j':'j.beta',
                          'HLA-A': 'mhc.a',
                          'HLA-A_1': 'mhc.a2',
                          'HLA-B':'mhc.b',
                          'HLA-B_1':'mhc.b2',
                          'HLA-C': 'mhc.c',
                          'HLA-C_1': 'mhc.c2',
                          'ORF Coverage': 'antigen.gene',
                          'Cohort': 'condition',
                          'Subject':'subject',
                                })
                                

In [79]:
mira['species']=['SARS-CoV-2']*len(mira)

In [94]:
mira['subject:condition']=[mira.iloc[i]['subject']+':'+mira.iloc[i]['condition'] for i in range(len(mira))]

In [95]:
mira.to_csv('data/MIRA_cleaned.csv')

In [117]:
mira=pd.read_csv('data/MIRA_cleaned.csv',index_col=0)

/var/folders/29/8tvn77zx2k39kgt070rqknwr0000gq/T/ipykernel_54665/1332532093.py:1: DtypeWarning: Columns (7,8,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  mira=pd.read_csv('data/MIRA_cleaned.csv',index_col=0)


In [118]:
from cluster_functions import load_vdjdb
import os
vdjdb = load_vdjdb(os.path.join(os.getcwd(),'data/vdjdb_full.txt'))

In [119]:
mira['antigen.species']=mira['species']

In [122]:
mira.to_csv('data/mira_cleaned.csv')

In [124]:
combined = pd.concat([vdjdb,mira])[vdjdb.columns]

In [126]:
combined=combined.replace(np.nan,'NA')

In [128]:
combined.to_csv('data/vdjdb_mira.csv')

In [21]:
for col in ['cdr3.alpha','v.alpha','j.alpha',
            'cdr3.beta','v.beta','j.beta']:
            print(col, set([type(x) for x in combined[col].unique()]))

cdr3.alpha {<class 'str'>, <class 'float'>}
v.alpha {<class 'str'>, <class 'float'>}
j.alpha {<class 'str'>, <class 'float'>}
cdr3.beta {<class 'str'>}
v.beta {<class 'str'>, <class 'float'>}
j.beta {<class 'str'>, <class 'float'>}


In [18]:
nans=[x for x in combined['cdr3.beta'].values if type(x) == float]
print(len(nans))

7020


In [23]:
combined=combined[~combined['cdr3.beta'].isin(np.concatenate([nans,vnans,jnans]))]

In [22]:
vnans, jnans = [[x for x in combined[col] if type(x)==float] for col in ['v.beta','j.beta']]
print(*[len(x) for x in [vnans,jnans]])

9 38


In [25]:
combined = combined.drop(columns=['cdr3.alpha','v.alpha','j.alpha'])

In [27]:
combined=combined.replace(np.nan,'NA')

In [28]:
combined.iloc[0]

cdr3.beta                         CASSYLPGQGDHYSNQPQHF
v.beta                                       TRBV13*01
d.beta                                              NA
j.beta                                      TRBJ1-5*01
species                                    HomoSapiens
mhc.a                                         HLA-B*08
mhc.b                                              B2M
mhc.class                                         MHCI
Epitope                    HLA-B*08_FLKEKGGL_Nef_HIV-1
antigen.gene                                       Nef
antigen.species                                  HIV-1
condition                       antigen-loaded-targets
subject                                            005
subject:condition                            005:vdjdb
Unnamed: 0                                          NA
Experiment                                          NA
Cell Type                                           NA
Target Type                                         NA
Age       

In [109]:
idx = [combined.iloc[i].name for i in range(len(combined)) if type(combined.iloc[i]['cdr3.beta'])==float]

In [112]:
combined.iloc[idx]

In [110]:
combined[combined.index.isin(idx)]

KeyboardInterrupt: 

In [29]:
combined.to_csv('data/vdjdb_mira.csv')

In [83]:
comb = pd.read_csv('data/combined_cdr3.csv')

In [69]:
cdr3s,eps=[mira[col].values for col in ['cdr3.beta','Epitope']]


In [70]:
hla,pep,gene,org = [[x.split('_')[i] for x in eps.tolist()] for i in range(4)]

In [71]:
eps_new = ['mira:'+'_'.join([org[i],gene[i],pep[i]]) for i in range(len(eps))]

In [74]:
mira_ref = pd.DataFrame([cdr3s,eps_new]).T
mira_ref.columns=['CDR3','Epitope']
mira_ref

,CDR3,Epitope
0,CASSFNPGHPNGYTF,mira:SARS-CoV-2_ORF1ab_TSDLATNNLVVMAY
1,CASRPPGYNEQFF,mira:SARS-CoV-2_ORF1ab_APKEIIFL
2,CASRPPGYNEQFF,mira:SARS-CoV-2_ORF1ab_KEIIFLEGETL
3,CASSLDGSGSYEQYF,mira:SARS-CoV-2_ORF1ab_YLKLTDNVYIK
4,CASSYMGFGAEAFF,"mira:SARS-CoV-2_ORF1ab,ORF3a_AEIPKEEVKPF"
...,...,...
630383,CASSSSGNTEAFF,mira:SARS-CoV-2_ORF8_YEDFLEYHDVRVVL
630384,CASSEVVSPGSSVRDEQFF,mira:SARS-CoV-2_nucleocapsid phosphoprotein_LS...
630385,CASSEVVSPGSSVRDEQFF,mira:SARS-CoV-2_nucleocapsid phosphoprotein_SP...
630386,CASSAAETSGSGETQYF,mira:SARS-CoV-2_nucleocapsid phosphoprotein_LS...


In [84]:
comb=pd.concat([comb,mira_ref])

In [86]:
comb=comb.drop(columns=['Unnamed: 0'])

In [96]:
comb=comb.replace(np.nan,'NA')

In [98]:
cdr3na, epna = [np.where(comb[x]=='NA') for x in ['CDR3','Epitope']]

In [100]:
comb=comb[~comb.index.isin(cdr3na+epna)]

In [102]:
comb

,CDR3,Epitope
0,CASSYSWGEQFF,giana:FNDC3B-VVL_L3M
1,CASTHWERVDEQFF,giana:OR6F1-VLN
2,CASSYGVNTEAFF,giana:OR14C36-FML_V6L
3,CAISRGDTEAFF,giana:TTLL12-KLP_N7D
4,CATSAKTRLNTEAFF,giana:ME1-FLD_A8G
...,...,...
630383,CASSSSGNTEAFF,mira:SARS-CoV-2_ORF8_YEDFLEYHDVRVVL
630384,CASSEVVSPGSSVRDEQFF,mira:SARS-CoV-2_nucleocapsid phosphoprotein_LS...
630385,CASSEVVSPGSSVRDEQFF,mira:SARS-CoV-2_nucleocapsid phosphoprotein_SP...
630386,CASSAAETSGSGETQYF,mira:SARS-CoV-2_nucleocapsid phosphoprotein_LS...


In [103]:
comb.to_csv('data/combined_cdr3_2.csv')

In [6]:
combined=pd.read_csv('data/vdjdb_mira.csv')

/var/folders/rn/d8fnv5qx1tn2vqzsj5c5dftw0000gn/T/ipykernel_5365/3066700457.py:1: DtypeWarning: Columns (1,2,3,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  combined=pd.read_csv('data/vdjdb_mira.csv')


In [11]:
combined=combined.dropna(subset=['cdr3.beta'])

In [23]:
alphabet=list(set(np.concatenate([[aa for aa in cdr3] for cdr3 in combined['cdr3.beta'].unique()])))

In [24]:
alphabet

['L',
 'G',
 'Y',
 'A',
 'E',
 'C',
 'T',
 'D',
 'H',
 'F',
 'O',
 'W',
 'K',
 'S',
 'U',
 'I',
 'Q',
 'V',
 'N',
 'P',
 'M',
 'R']

In [17]:
oovs = [x for x in combined['cdr3.beta'].values if ('*' in x)|('U' in x)]
combined = combined[~combined['cdr3.beta'].isin(oovs)]

In [21]:
combined['cdr3.beta']=[x.upper() for x in combined['cdr3.beta'].values]
combined['cdr3.alpha']=[x if type(x) == float else x.upper() for x in combined['cdr3.alpha'].values]

In [22]:
combined.to_csv('data/vdjdb_mira.csv')

In [44]:
vdjdb=[ep for ep in comb['Epitope'] if 'vdjdb' in ep]
comb[comb['Epitope'].isin(vdjdb)]

,Unnamed: 0,CDR3,Epitope
61367,0,CIVRAPGRADMRF,vdjdb: HIV-1-Nef_FLKEKGGL
61368,1,NaN,vdjdb: HIV-1-Nef_FLKEKGGL
61369,2,CAVPSGAGSYQLTF,vdjdb: HIV-1-Nef_FLKEKGGL
61370,3,CAVKASGSRLT,vdjdb: HIV-1-Nef_FLKEKGGL
61371,4,CAYRPPGTYKYIF,vdjdb: HIV-1-Nef_FLKEKGGL
...,...,...,...
158710,53576,CSARGGAVFYEQYF,vdjdb: HIV-1-Gag_KRWIIMGLNK
158711,53577,CASSDLTGTVYNEQFF,vdjdb: HIV-1-Gag_KRWIIMGLNK
158712,53578,CSAWGGSVFYEQYF,vdjdb: HIV-1-Gag_KRWIIMGLNK
158713,53579,CSARGRERFYEQYF,vdjdb: HIV-1-Gag_KRWIIMGLNK


In [34]:
comb.iloc[0]

Unnamed: 0                       0
CDR3                  CASSYSWGEQFF
Epitope       giana:FNDC3B-VVL_L3M
Name: 0, dtype: object

In [ ]:
tcra,tcrb = [tcrs[x].values.tolist() for x in ['cdr3.alpha','cdr3.beta']]

In [ ]:
reftest = [tcr in refs['CDR3'] for tcr in tcrs]

In [ ]:
mira = pd.read_csv('/Users/danhudson/Documents/Academic/Oxford/Oxford_DPhil/Rotation_Projects/Hashem/clusTCR/tcr_compare/data/MIRA_trimmed3.csv',index_col=0) 


In [ ]:
refbeta = [tcr in mira['cdr3.beta'] for tcr in tcrb]
True in refbeta

In [ ]:
nodelist = '/Users/danhudson/Documents/Academic/Oxford/Oxford_DPhil/Software/clusTCR/tcr_compare/cytoscape/20220503_133906_ORF3a_data.csv/tcrdist3_paired_ORF3a_data.csv_nodelist.txt'

In [ ]:
nodes = pd.read_csv(nodelist)

In [ ]:
nodecheckbeta= [tcr in nodes['CDR3'].values for tcr in tcrb]
nodecheckalpha= [tcr in nodes['CDR3'].values for tcr in tcra]
covered = zip(nodecheckalpha,nodecheckbeta)

In [ ]:
list(covered)

In [ ]:
out=[]
for i, tcrcheck in enumerate(list(covered)):
    print(tcrcheck)
    if True in tcrcheck:
        out.append(nodes.iloc[i]['cluster'])
    else:
        out.append('NA')
tcrs['cluster'] = out


In [ ]:
tcrs['cluster']

In [ ]:
nodes[nodes['cluster']==436]['Epitope'].value_counts()

In [ ]:
subset=nodes[nodes['CDR3'].isin(tcrb)]

eps_arr=dict()
for i,cdr3 in enumerate(subset['CDR3'].unique()):
    cluster = subset.iloc[i]['cluster']
    if cluster=='NA':
        eps_arr[i]='NA'
    else:
        eps = nodes[nodes['cluster']==cluster]['Epitope'].value_counts()
        eps_arr[i]=eps



In [ ]:
tcrs['cluster_epcounts']=tcrs[eps_arr


In [ ]:
subset.iloc[0]['cluster_epitopes']()